### Building and Running a Graph

In [3]:
import tensorflow as tf

In [2]:
# two tensor constants
node1 = tf.constant(30, tf.float32)
node2 = tf.constant(4.0)

print(node1, node2) # outputs an abstract tensor

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


#### Creating Session

In [3]:
sess = tf.Session() # use resources such as CPU or GPU
print(sess.run([node1, node2]))
sess.close() # free up resources

[30.0, 4.0]


#### Other way to create session

In [6]:
with tf.Session() as sess:
    output = sess.run([node1, node2])
    print(output)

[30.0, 4.0]


### Visualizing computational graph using Tensorboard

This can be done by first creating a tf.summary.FileWriter object, passing the path of the directory where the information will be store that can be use by Tensorboard to create the graph

In [9]:
help(tf.summary.FileWriter)

Help on class FileWriter in module tensorflow.python.summary.writer.writer:

class FileWriter(SummaryToEventTransformer)
 |  Writes `Summary` protocol buffers to event files.
 |  
 |  The `FileWriter` class provides a mechanism to create an event file in a
 |  given directory and add summaries and events to it. The class updates the
 |  file contents asynchronously. This allows a training program to call methods
 |  to add data to the file directly from the training loop, without slowing down
 |  training.
 |  
 |  Method resolution order:
 |      FileWriter
 |      SummaryToEventTransformer
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, logdir, graph=None, max_queue=10, flush_secs=120, graph_def=None, filename_suffix=None)
 |      Creates a `FileWriter` and an event file.
 |      
 |      On construction the summary writer creates a new event file in `logdir`.
 |      This event file will contain `Event` protocol buffers constructed when you
 |      cal

In [2]:
a = tf.constant(7.0)
b = tf.constant(5.0)

c = a * b

sess = tf.Session()

File_Writer = tf.summary.FileWriter('graph', sess.graph)

print(sess.run(c))

sess.close()

35.0


After executing the above code the present working directory will have a folder name **graph**. 
Now we need to go to the terminal and type: 
```bash 
tensorflow --logdir='graph' 
TensorBoard 0.1.5 at http://baka:6006 (Press CTRL+C to quit) 
```

Visit http://baka:6006. The graph looks something like this:

![Initial Graph](initial_graph.png)

#### Placeholder

In [4]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

adder_node = a + b

sess = tf.Session()

print(sess.run(adder_node, {a:[4, 5], b:[2, 7]}))

sess.close()

[  6.  12.]


#### Variables

In [15]:
sess = tf.Session()

# Model Parameters
W = tf.Variable([0.0], tf.float32)
b = tf.Variable([0.0], tf.float32)

# Inputs and Outputs
x = tf.placeholder(tf.float32)

linear_model = W * x + b

y = tf.placeholder(tf.float32)

# Loss function
squared_error = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_error)

# Optimizer
optimizer = tf.train.GradientDescentOptimizer(.01)
train = optimizer.minimize(loss)



# This will intialize the global variables
init = tf.global_variables_initializer()

sess.run(init) #Initializing global variables

# print(sess.run(loss, {x:[1, 2, 3, 4], y:[5, 8, 11, 14]}))

for i in range(1000):
    sess.run(train, {x:[1, 2, 3, 4], y:[5, 8, 11, 14]})

print(sess.run([W, b]))

sess.close()

[array([ 3.00000238], dtype=float32), array([ 1.99999309], dtype=float32)]


### Implementation of Naval Mine Identifier